In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem

In [2]:
from scipy import stats

In [3]:
import pandas as pd, numpy as np, re, os
from src.evaluation import Evaluator, ResultsFromMeanStdDF
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 12})

In [4]:
## This cell would need for testing phase - Ignore this
%load_ext autoreload
%autoreload 2

In [5]:
paths = [ os.path.join(dataset.name, experiment.name) 
         for dataset in os.scandir('experiments') if dataset.is_dir() and 'chembl' in dataset.name 
         for experiment in os.scandir(dataset.path)  ]

In [6]:
paths=[]
for dataset in os.scandir('experiments'):
    if not dataset.is_dir() or not 'atom-fragSMILES' in dataset.name:
        continue

    if 'chemblAug5' in dataset.name:
        for setup in os.scandir(dataset):
            if '2hl_512hu' in setup.name and '256bs' in setup.name:
                paths.append(setup.path)

    elif 'chembl' in dataset.name:
        for setup in os.scandir(dataset):
            if '2hl_512hu' in setup.name and '512bs' in setup.name:
                paths.append(setup.path)

In [7]:
paths

['experiments/chembl_atom-fragSMILES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chemblAug5_atom-fragSMILES-RNN/2hl_512hu_256bs_0.001lr']

In [8]:
evaluators = [Evaluator(path) for path in paths]

In [9]:
evaluators_table = pd.concat( [evaluator.get_as_DFcell() for evaluator in evaluators ], ignore_index=True, axis=0)

In [10]:
evaluators_table = evaluators_table.drop(columns=['dataset','notation']).groupby(['aug','hl','hu','bs','lr']).agg(lambda x: x.dropna())

In [11]:
evaluators_table

,,,,,atom-fragSMILES
aug,hl,hu,bs,lr,
1,2,512,512,0.001,<src.evaluation.Evaluator object at 0x7f735df4...
5,2,512,256,0.001,<src.evaluation.Evaluator object at 0x7f74d018...


In [12]:
evaluators_table.map(lambda x: x.load_novels())

,,,,,atom-fragSMILES
aug,hl,hu,bs,lr,
1,2,512,512,0.001,True
5,2,512,256,0.001,True


In [13]:
evaluators_table.map(lambda x: x.load_scaffolds())

,,,,,atom-fragSMILES
aug,hl,hu,bs,lr,
1,2,512,512,0.001,True
5,2,512,256,0.001,True


In [14]:
drop_cols = ['fold','epoch','T','amount','dataset']
group_cols = ['aug','notation','hl','hu','bs','lr']

## Novel metric

In [15]:
novels = pd.concat( [evaluator.getResultsNovels() for evaluator in evaluators], ignore_index=True)

In [16]:
novels

,FCD/Test,SNN/Test,Scaf/Test,IntDiv,IntDiv2,Filters,logP,SA,QED,weight,...,aug,notation,hl,hu,bs,lr,fold,amount,epoch,T
0,1.150073,0.484608,0.388104,0.869483,0.863729,0.853500,0.197486,0.137013,0.006273,11.906130,...,1,atom-fragSMILES,2,512,512,0.001,0,6000,12,1.0
1,0.906940,0.485669,0.371099,0.868817,0.863145,0.847667,0.053088,0.073055,0.009429,4.477463,...,1,atom-fragSMILES,2,512,512,0.001,1,6000,12,1.0
2,0.988004,0.471207,0.341589,0.868088,0.862615,0.846833,0.089135,0.051500,0.014902,2.285263,...,1,atom-fragSMILES,2,512,512,0.001,2,6000,12,1.0
3,0.863898,0.482299,0.388123,0.867481,0.861920,0.861667,0.052505,0.060340,0.008239,5.262249,...,1,atom-fragSMILES,2,512,512,0.001,3,6000,12,1.0
4,1.146447,0.486721,0.380045,0.867692,0.861964,0.846833,0.200858,0.116674,0.008704,7.855205,...,1,atom-fragSMILES,2,512,512,0.001,4,6000,12,1.0
5,0.849130,0.483491,0.399226,0.867840,0.862374,0.845667,0.052330,0.071790,0.005426,8.155338,...,5,atom-fragSMILES,2,512,256,0.001,0,6000,4,1.0
6,0.906077,0.480123,0.389402,0.871093,0.865457,0.853000,0.032232,0.046796,0.007501,5.979900,...,5,atom-fragSMILES,2,512,256,0.001,1,6000,4,1.0
7,0.973573,0.475634,0.315701,0.866623,0.861142,0.842500,0.308993,0.039660,0.046388,13.554268,...,5,atom-fragSMILES,2,512,256,0.001,2,6000,4,1.0
8,0.950941,0.478989,0.411039,0.867427,0.861952,0.846167,0.080376,0.037518,0.033190,6.061926,...,5,atom-fragSMILES,2,512,256,0.001,3,6000,4,1.0
9,0.936929,0.476496,0.374817,0.867290,0.861895,0.853333,0.174071,0.065644,0.027474,9.580440,...,5,atom-fragSMILES,2,512,256,0.001,4,6000,4,1.0


In [17]:
novGrp=novels.drop(columns=drop_cols)\
        .groupby(group_cols, as_index=True)

In [18]:
novStats=novGrp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ]

In [19]:
novStats

FCD/Test            SNN/Test            \
                                          mean       std      mean       std   
aug notation        hl hu  bs  lr                                              
1   atom-fragSMILES 2  512 512 0.001  1.011072  0.132932  0.482101  0.006306   
5   atom-fragSMILES 2  512 256 0.001  0.923330  0.048153  0.478947  0.003123   

                                     Scaf/Test              IntDiv            \
                                          mean       std      mean       std   
aug notation        hl hu  bs  lr                                              
1   atom-fragSMILES 2  512 512 0.001  0.373792  0.019324  0.868312  0.000829   
5   atom-fragSMILES 2  512 256 0.001  0.378037  0.037294  0.868054  0.001754   

                                       IntDiv2             Filters            \
                                          mean       std      mean       std   
aug notation        hl hu  bs  lr                                              
1   atom-fragSMILES 2  512 512 0.001  0.862675  0.000776  0.851300  0.006431   
5   atom-fragSMILES 2  512 256 0.001  0.862564  0.001677  0.848133  0.004807   

                                          logP                  SA            \
                                          mean       std      mean       std   
aug notation        hl hu  bs  lr                                              
1   atom-fragSMILES 2  512 512 0.001  0.118614  0.075030  0.087716  0.037231   
5   atom-fragSMILES 2  512 256 0.001  0.129600  0.114054  0.052282  0.015544   

                                           QED              weight            
                                          mean       std      mean       std  
aug notation        hl hu  bs  lr                                             
1   atom-fragSMILES 2  512 512 0.001  0.009509  0.003233  6.357262  3.686042  
5   atom-fragSMILES 2  512 256 0.001  0.023996  0.017428  8.666374  3.122037

## Scaffold

In [20]:
scaffDFs=pd.concat( [evaluator.getScaffoldResults() for evaluator in evaluators if not evaluator.novels is None], ignore_index=True)

In [21]:
scaffDFs['unique%']=scaffDFs['unique']/scaffDFs['total']
scaffDFs['novel%']=scaffDFs['novel']/scaffDFs['unique']
scaffDFs['chiral%']=scaffDFs['chiral']/scaffDFs['novel']

In [22]:
scaffDFs

,fold,amount,epoch,T,total,unique,novel,chiral,dataset,aug,notation,hl,hu,bs,lr,unique%,novel%,chiral%
0,0,6000,12,1.0,5982,5524,4652,546,chembl,1,atom-fragSMILES,2,512,512,0.001,0.923437,0.842143,0.117369
1,1,6000,12,1.0,5970,5504,4607,525,chembl,1,atom-fragSMILES,2,512,512,0.001,0.921943,0.837028,0.113957
2,2,6000,12,1.0,5987,5596,4777,574,chembl,1,atom-fragSMILES,2,512,512,0.001,0.934692,0.853645,0.120159
3,3,6000,12,1.0,5980,5500,4658,586,chembl,1,atom-fragSMILES,2,512,512,0.001,0.919732,0.846909,0.125805
4,4,6000,12,1.0,5975,5481,4549,553,chembl,1,atom-fragSMILES,2,512,512,0.001,0.917322,0.829958,0.121565
5,0,6000,4,1.0,5987,5467,4537,515,chembl,5,atom-fragSMILES,2,512,256,0.001,0.913145,0.829888,0.113511
6,1,6000,4,1.0,5969,5477,4517,638,chembl,5,atom-fragSMILES,2,512,256,0.001,0.917574,0.824722,0.141244
7,2,6000,4,1.0,5988,5605,4766,546,chembl,5,atom-fragSMILES,2,512,256,0.001,0.936039,0.850312,0.114561
8,3,6000,4,1.0,5979,5431,4493,573,chembl,5,atom-fragSMILES,2,512,256,0.001,0.908346,0.827288,0.127532
9,4,6000,4,1.0,5981,5515,4548,649,chembl,5,atom-fragSMILES,2,512,256,0.001,0.922087,0.824660,0.142700


## New frags

In [23]:
from src.processer import fragmentSmiles, applyFncPool

In [24]:
vocabs=[]
train_data = pd.read_csv(f'data/chembl.tar.xz', usecols = ['fragsmiles'] + [f'fold{fold}' for fold in range(5)], compression="xz")
train_data['fragsmiles'] = train_data['fragsmiles'].str.split(' ')
for fold in range(5):
    query = f'fold{fold} == "train"'
    fold_data = train_data.query(query)['fragsmiles']
    vocabs.append(np.unique(np.concatenate(fold_data.values)))

In [25]:
def checkScaffoldFragments(scaffolds, vocab):

    def checkNewFrag(frags):
        mask = np.in1d(frags, vocab, invert=True)

        ## If u want track which new fragments are
        # new = np.array(frags)[mask]
        # if len(new)>0:
        #     print(new)
        return mask.any()
    
    scaffFrags = pd.Series( applyFncPool(scaffolds, fnc=fragmentSmiles) ).str.split(' ')
    maskNewFrags=scaffFrags.apply(checkNewFrag)

    # return maskNewFrags
    return maskNewFrags.sum()


In [26]:
def countNewScaffCauseFrags(evaluator):
    results = evaluator.novels.copy()

    results['inducedNewFrags']=results.apply(lambda x: checkScaffoldFragments(x.sampled.novelScaff, vocabs[x.fold]), axis=1)
    # return results.apply(lambda x: checkScaffoldFragments(x.sampled.novelScaff, vocabs[x.fold]), axis=1)

    setup = {k:v for k,v in evaluator.setupArgs.items() if k!='es'}
    results = results.assign(**evaluator.datasetArgs, **setup)

    results.drop(columns='sampled', inplace=True)
    return results.sort_values(['amount','fold','epoch'])

In [27]:
fragInduced = pd.concat( [ countNewScaffCauseFrags(evaluator) for evaluator in evaluators if evaluator.datasetArgs['notation']!='fragSMILES'], ignore_index=True)

In [28]:
fragInduced

,fold,amount,epoch,T,inducedNewFrags,dataset,aug,notation,hl,hu,bs,lr
0,0,6000,12,1.0,163,chembl,1,atom-fragSMILES,2,512,512,0.001
1,1,6000,12,1.0,172,chembl,1,atom-fragSMILES,2,512,512,0.001
2,2,6000,12,1.0,176,chembl,1,atom-fragSMILES,2,512,512,0.001
3,3,6000,12,1.0,174,chembl,1,atom-fragSMILES,2,512,512,0.001
4,4,6000,12,1.0,146,chembl,1,atom-fragSMILES,2,512,512,0.001
5,0,6000,4,1.0,124,chembl,5,atom-fragSMILES,2,512,256,0.001
6,1,6000,4,1.0,147,chembl,5,atom-fragSMILES,2,512,256,0.001
7,2,6000,4,1.0,130,chembl,5,atom-fragSMILES,2,512,256,0.001
8,3,6000,4,1.0,113,chembl,5,atom-fragSMILES,2,512,256,0.001
9,4,6000,4,1.0,139,chembl,5,atom-fragSMILES,2,512,256,0.001


## Rings

In [29]:
ringsDFs=pd.concat( [evaluator.getRingsResults() for evaluator in evaluators if not evaluator.novels is None], ignore_index=True)

In [30]:
ringsDFs['unique%']=ringsDFs['unique']/ringsDFs['total']
ringsDFs['novel%']=ringsDFs['novel']/ringsDFs['unique']
ringsDFs['chiral%']=ringsDFs['chiral']/ringsDFs['novel']

In [31]:
ringsDFs

,fold,amount,epoch,T,total,unique,novel,chiral,dataset,aug,notation,hl,hu,bs,lr,unique%,novel%,chiral%
0,0,6000,12,1.0,18189,824,164,8,chembl,1,atom-fragSMILES,2,512,512,0.001,0.045302,0.199029,0.048780
1,1,6000,12,1.0,18156,844,174,8,chembl,1,atom-fragSMILES,2,512,512,0.001,0.046486,0.206161,0.045977
2,2,6000,12,1.0,18487,822,169,5,chembl,1,atom-fragSMILES,2,512,512,0.001,0.044464,0.205596,0.029586
3,3,6000,12,1.0,17995,836,174,9,chembl,1,atom-fragSMILES,2,512,512,0.001,0.046457,0.208134,0.051724
4,4,6000,12,1.0,18388,777,145,4,chembl,1,atom-fragSMILES,2,512,512,0.001,0.042256,0.186615,0.027586
5,0,6000,4,1.0,18280,826,123,4,chembl,5,atom-fragSMILES,2,512,256,0.001,0.045186,0.148910,0.032520
6,1,6000,4,1.0,17959,876,140,4,chembl,5,atom-fragSMILES,2,512,256,0.001,0.048778,0.159817,0.028571
7,2,6000,4,1.0,19046,833,133,5,chembl,5,atom-fragSMILES,2,512,256,0.001,0.043736,0.159664,0.037594
8,3,6000,4,1.0,17715,852,116,3,chembl,5,atom-fragSMILES,2,512,256,0.001,0.048095,0.136150,0.025862
9,4,6000,4,1.0,18237,856,136,5,chembl,5,atom-fragSMILES,2,512,256,0.001,0.046938,0.158879,0.036765


## Merged

In [37]:
grp = novels.merge(scaffDFs).merge(fragInduced).drop(columns=['dataset','T','fold','amount','epoch'])\
                    .groupby(['aug','notation','hl','hu','bs','lr'], as_index=True, dropna=False)

In [45]:
ResultsFromMeanStdDF (
    grp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ]
).to_csv('SI_tables/atom-tokenized_fragSMILES_metrics.csv',mode='a')

In [43]:
grp_rings = ringsDFs.drop(columns=['dataset','T','fold','amount','epoch'])\
                    .groupby(['aug','notation','hl','hu','bs','lr'], as_index=True, dropna=False)

In [46]:
ResultsFromMeanStdDF (
    grp_rings.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ]
).to_csv('SI_tables/atom-tokenized_fragSMILES_rings_metrics.csv',mode='a')